In [4]:
import pyspark
import os

os.environ['SPARK_DIST_CLASSPATH']="""/opt/spark2/conf/yarn-conf:/opt/hadoop2/share/hadoop/common/lib/*:/opt/hadoop2/share/hadoop/common/*:/opt/hadoop2/share/hadoop/hdfs:/opt/hadoop2/share/hadoop/hdfs/lib/*:/opt/hadoop2/share/hadoop/hdfs/*:/opt/hadoop2/share/hadoop/yarn/lib/*:/opt/hadoop2/share/hadoop/yarn/*:/opt/hadoop2/share/hadoop/mapreduce/lib/*:/opt/hadoop2/share/hadoop/mapreduce/*"""


from pyspark import SparkContext, SparkConf, SQLContext

from pyspark.streaming import StreamingContext

from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [8]:
spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

In [11]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

In [15]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()


query.awaitTermination(10)

False

In [ ]:
#./bin/spark-submit examples/src/main/python/sql/streaming/structured_network_wordcount.py localhost 9999